In [157]:
# load some library needed and functions

import os
import json
import sounddevice as sd
import soundfile as sf
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
import lbg

# Definition of Variables


In [178]:
# Audio constants
duration_s = 2
fs = 44100
## Change variables for optimization 
# MFCC
num_mfcc = 13
frame_lenght = 1024
hop_lenght = 512
#Codebook
cb_depth = 8
thr = 8000
#TODO threshold

mfcc = [7, 10]
cb_depths = [16, 32]

In [159]:
def record_audio(seconds, user_name, entry):
    fs = 44100  # Sampling frequency
    duration = seconds  # Duration of recording in seconds
    
    
    file.write("Recording...")

    
    # Record audio from the microphone
    audio = sd.rec(int(duration * fs), samplerate=fs, channels=1, dtype=np.float32)
    sd.wait()

    file.write("Finished recording.")

    if user_name:
        filename = f"voice_{user_name}_{entry}.wav"
        folder_path = os.path.join(os.getcwd(), 'Users_samples', user_name, filename)
        # Save audio to file
        sf.write(folder_path, audio, fs, subtype='PCM_24')
        file.write(f"Audio recorded and saved as {folder_path}")
    
    
    return audio, fs


def extract_mfcc_file(audio_file, num_mfcc, frame_length, hop_length):
    # Load audio file
    y, sr = librosa.load(audio_file)

    # Extract MFCCs
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=num_mfcc, n_fft=frame_length, hop_length=hop_length)

    return mfccs

def extract_mfcc(audio_waveform, sr, num_mfcc, frame_length, hop_length):
    # Extract MFCCs
    mfccs = librosa.feature.mfcc(y=audio_waveform, sr=sr, n_mfcc=num_mfcc, n_fft=frame_length, hop_length=hop_length)

    return mfccs

def calculate_distortion(sample_mfcc, codebook):
    # Initialize total distortion
    total_distortion = 0.0
    
    # Assign each sample MFCC to its nearest centroid in the codebook
    nearest_centroids = np.argmin(np.linalg.norm(sample_mfcc[:, None, :] - codebook, axis=2), axis=1)
    
    # Calculate the Euclidean distance between each sample MFCC and its assigned centroid
    for i, centroid_index in enumerate(nearest_centroids):
        centroid = codebook[centroid_index]
        distortion = np.linalg.norm(sample_mfcc[i] - centroid)
        total_distortion += distortion
        # for average distortion, divide the total by sample_mfcc.shape[0]
    return total_distortion


def create_user_folder(user_name):
    # Create folder path
    user_folder='Users_samples'
    folder_path = os.path.join(os.getcwd(), user_folder, user_name)

    # Check if folder exists
    if not os.path.exists(folder_path):
        # Create folder if it doesn't exist
        os.makedirs(folder_path)
        print(f"Folder '{user_name}' created successfully.")
    else:
        file.write(f"Folder '{user_name}' already exists.")
    
    file_count = len([name for name in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, name))])
    return file_count, folder_path

def scale_number(unscaled, to_min, to_max, from_min, from_max):
    return (to_max-to_min)*(unscaled-from_min)/(from_max-from_min)+to_min

def scale_list(l, to_min, to_max):
    return [scale_number(i, to_min, to_max, min(l), max(l)) for i in l]


# Add users to database and create filesystem

In [160]:
# user_name = input("Enter your name: ")
# entry, folder = create_user_folder(user_name)

# audio_file, fs = record_audio(duration_s, user_name, entry+1)
# audio = audio_file.reshape(-1)

In [180]:
for num_mfcc in mfcc:
    for cb_depth in cb_depths:
        # Create codebooks
        user_folder='Users_samples'
        users_folder= os.path.join(os.getcwd(), user_folder)

        codebook_folder_rel='Codebooks'
        codebook_folder= os.path.join(os.getcwd(), codebook_folder_rel)

        folder_names = [name for name in os.listdir(users_folder) if os.path.isdir(os.path.join(users_folder, name))]


        audio_per_user = {}
        mfccs_per_user = {}
        codebook_per_user = {}

        for folder_name in folder_names:
            folder_path = os.path.join(users_folder, folder_name)
            # Check if folder exists
            if not os.path.exists(folder_path) or not os.path.isdir(folder_path):
                print(f"Folder '{folder_name}' does not exist or is not a directory. Skipping.")
                continue

            # Read audio files in the folder
            concatenated_audio = []
            for filename in os.listdir(folder_path):
                file_path = os.path.join(folder_path, filename)

                # Check if file is an audio file
                if filename.endswith('.wav') or filename.endswith('.mp3'):
                    print(f"Reading audio file: {filename}")
                    audio, sr = librosa.load(file_path, sr=fs)
                    normalized_audio = audio / np.max(np.abs(audio))
                    concatenated_audio.extend(audio)


            audio_per_user[folder_name] = concatenated_audio
        
        for user in audio_per_user:
            num_mfcc = num_mfcc  # Number of MFCC coefficients, global variable
            frame_length = frame_lenght  # Frame length for STFT
            hop_length = hop_lenght  # Hop length for STFT
            audio_user=np.array(audio_per_user[user])
            
            mfccs_per_user[user] = extract_mfcc(audio_user, fs, num_mfcc, frame_length, hop_length)
            cb, cb_abs_w, cb_rel_w = lbg.generate_codebook(mfccs_per_user[user].T, cb_depth)
            codebook_per_user[user] = cb
            
        with open(os.path.join(codebook_folder, 'Codebook'), 'w') as f:
            json.dump(codebook_per_user, f)  
            
        print("Finished creating Codebook.")    







        #Read codebooks and identify speaker
        sample_filenames = ['Katia_0.wav', 'Katia_1.wav', 'Katia_2.wav', 'Katia_3.wav', 'Katia_4.wav', 'Katia_5.wav', 'Katia_6.wav', 'Katia_7.wav', 'Katia_8.wav', 'Katia_9.wav', 'Facundo_0.wav', 'Facundo_1.wav', 'Facundo_2.wav', 'Facundo_3.wav', 'Facundo_4.wav', 'Facundo_5.wav', 'Facundo_6.wav', 'Facundo_7.wav', 'Facundo_8.wav', 'Facundo_9.wav','Cesar_0.wav', 'Cesar_1.wav', 'Cesar_2.wav', 'Cesar_3.wav', 'Cesar_4.wav', 'Cesar_5.wav', 'Cesar_6.wav', 'Cesar_7.wav', 'Cesar_8.wav', 'Cesar_9.wav', 'Alanny_0.wav', 'Alanny_1.wav', 'Alanny_2.wav', 'Alanny_3.wav', 'Alanny_4.wav', 'Alanny_5.wav', 'Alanny_6.wav', 'Alanny_7.wav', 'Alanny_8.wav', 'Alanny_9.wav']
        sample_folder_rel='Sample'
        sample_folder= os.path.join(os.getcwd(), sample_folder_rel)
        print(f"mfcc\t{num_mfcc}")
        print(f"codebook depth\t{cb_depth}")
        print("----------------")
        print("----------------")
        with open('results.txt', 'a') as file:
            file.write(f"mfcc\t{num_mfcc}\n")
            file.write(f"codebook depth\t{cb_depth}\n")
            file.write("----------------\n")
        for filename in sample_filenames:
            sample_path= os.path.join(sample_folder, filename)
            audio_file, sr = librosa.load(sample_path, sr=fs)
            print(filename)
            with open('results.txt', 'a') as file:
                file.write(f'{filename}\n')
            # entry = input("Press a key to enter a voice sample for identification: ")
            # audio_file, fs = record_audio(2, "", "")

            audio = audio_file.reshape(-1)

            num_mfcc = num_mfcc  # Number of MFCC coefficients
            frame_length = frame_lenght  # Frame length for STFT
            hop_length = hop_lenght  # Hop length for STFT

            mfccs = extract_mfcc(audio, fs, num_mfcc, frame_length, hop_length)

            print("Loading Codebook ... ")

            with open(os.path.join(codebook_folder, 'Codebook'), 'r') as f:
                    stored_codebook = json.load(f)

            # calculate distortions

            print("User\tDistortion")
            print("----------------")
            user_distortion = {}
            for user, cb in stored_codebook.items():
                distortion = calculate_distortion(mfccs.T, cb)
                # print(f"{user}\t{distortion}")
                # with open('results.txt', 'a') as file:
                #     file.write(f"{user}\t{distortion}\n")
                if user not in user_distortion:
                    user_distortion[user] = distortion
            min_key = min(user_distortion, key=user_distortion.get)
            print(min_key)
            print(user_distortion.get(min_key))
            print("----------------")       
            with open('results.txt', 'a') as file:
                file.write(f'{min_key}\t{user_distortion.get(min_key)}\n')
                file.write("----------------\n")
                
        with open('results.txt', 'a') as file:
            file.write('---------------------------------------\n')
        





        
        # Plot
        feature1_idx = 1  
        feature2_idx = 2

        plt.figure(figsize=(12, 8)) 

        for user, sample_mfcc in mfccs_per_user.items():
            # Plot MFCC
            #plt.scatter(sample_mfcc[feature1_idx,:], sample_mfcc[feature2_idx,:], label=user)
            # Plot centroid
            cb_stored=np.array(stored_codebook[user])
            plt.scatter(cb_stored[:,feature1_idx], cb_stored[:,feature2_idx], label=f'Centroid of {user}', marker='x')
        
        plt.xlabel(f'Feature {feature1_idx+1}')
        plt.ylabel(f'Feature {feature2_idx+1}')
        plt.title(f'MFCC and Centroid')
        plt.legend()
        plt.grid(True)
        plt.savefig(f"mfcc_{num_mfcc}_cb_{cb_depth}.png")
        plt.close()
        plt.show()
            
        

Reading audio file: voice_Cesar_1.wav
Reading audio file: voice_Cesar_2.wav
Reading audio file: voice_Cesar_3.wav
Reading audio file: voice_Facundo_1.wav
Reading audio file: voice_Facundo_2.wav
Reading audio file: voice_Facundo_3.wav
Reading audio file: voice_Katia_1.wav
Reading audio file: voice_Katia_2.wav
Reading audio file: voice_Katia_3.wav
Finished creating Codebook.
mfcc	7
codebook depth	16
----------------
----------------
Katia_0.wav
Loading Codebook ... 
User	Distortion
----------------
Katia
3851.1256263918954
----------------
Katia_1.wav
Loading Codebook ... 
User	Distortion
----------------
Cesar
6291.078071581792
----------------
Katia_2.wav
Loading Codebook ... 
User	Distortion
----------------
Katia
3924.467711783021
----------------
Katia_3.wav
Loading Codebook ... 
User	Distortion
----------------
Cesar
6282.604020218162
----------------
Katia_4.wav
Loading Codebook ... 
User	Distortion
----------------
Katia
4362.575457734039
----------------
Katia_5.wav
Loading Code

# Create codebooks

In [162]:
# user_folder='Users_samples'
# users_folder= os.path.join(os.getcwd(), user_folder)

# codebook_folder_rel='Codebooks'
# codebook_folder= os.path.join(os.getcwd(), codebook_folder_rel)

# folder_names = [name for name in os.listdir(users_folder) if os.path.isdir(os.path.join(users_folder, name))]


# audio_per_user = {}
# mfccs_per_user = {}
# codebook_per_user = {}

# for folder_name in folder_names:
#     folder_path = os.path.join(users_folder, folder_name)
#     # Check if folder exists
#     if not os.path.exists(folder_path) or not os.path.isdir(folder_path):
#         print(f"Folder '{folder_name}' does not exist or is not a directory. Skipping.")
#         continue

#     # Read audio files in the folder
#     concatenated_audio = []
#     for filename in os.listdir(folder_path):
#         file_path = os.path.join(folder_path, filename)

#         # Check if file is an audio file
#         if filename.endswith('.wav') or filename.endswith('.mp3'):
#             print(f"Reading audio file: {filename}")
#             audio, sr = librosa.load(file_path, sr=fs)
#             normalized_audio = audio / np.max(np.abs(audio))
#             concatenated_audio.extend(audio)


#     audio_per_user[folder_name] = concatenated_audio
  
# for user in audio_per_user:
#     num_mfcc = num_mfcc  # Number of MFCC coefficients, global variable
#     frame_length = frame_lenght  # Frame length for STFT
#     hop_length = hop_lenght  # Hop length for STFT
#     audio_user=np.array(audio_per_user[user])
    
#     mfccs_per_user[user] = extract_mfcc(audio_user, fs, num_mfcc, frame_length, hop_length)
#     cb, cb_abs_w, cb_rel_w = lbg.generate_codebook(mfccs_per_user[user].T, cb_depth)
#     codebook_per_user[user] = cb
    
# with open(os.path.join(codebook_folder, 'Codebook'), 'w') as f:
#     json.dump(codebook_per_user, f)  
    
# print("Finished creating Codebook.")    

Reading audio file: voice_Cesar_1.wav
Reading audio file: voice_Cesar_2.wav
Reading audio file: voice_Cesar_3.wav
Reading audio file: voice_Facundo_1.wav
Reading audio file: voice_Facundo_2.wav
Reading audio file: voice_Facundo_3.wav
Reading audio file: voice_Katia_1.wav
Reading audio file: voice_Katia_2.wav
Reading audio file: voice_Katia_3.wav


KeyboardInterrupt: 

# Read the codebook and identify speaker

In [172]:
# sample_filenames = ['Alanny.wav', 'Facundo.wav', 'Cesar.wav', 'Katia.wav']
# sample_folder_rel='Sample'
# sample_folder= os.path.join(os.getcwd(), sample_folder_rel)
# print(f"mfcc\t{num_mfcc}")
# print(f"codebook depth\t{cb_depth}")
# # print("----------------")
# # print("----------------")
# # with open('results.txt', 'a') as file:
# #     file.write(f"mfcc\t{num_mfcc}\n")
# #     file.write(f"codebook depth\t{cb_depth}\n")
# #     file.write("----------------\n")
# #     file.write("----------------\n")
# for filename in sample_filenames:

#     sample_path= os.path.join(sample_folder, filename)
#     audio_file, sr = librosa.load(sample_path, sr=fs)
#     print(filename)
#     # with open('results.txt', 'a') as file:
#     #     file.write(f'{filename}\n')
#     # entry = input("Press a key to enter a voice sample for identification: ")
#     # audio_file, fs = record_audio(2, "", "")

#     audio = audio_file.reshape(-1)

#     num_mfcc = num_mfcc  # Number of MFCC coefficients
#     frame_length = frame_lenght  # Frame length for STFT
#     hop_length = hop_lenght  # Hop length for STFT

#     mfccs = extract_mfcc(audio, fs, num_mfcc, frame_length, hop_length)

#     # print("Loading Codebook ... ")

#     with open(os.path.join(codebook_folder, 'Codebook'), 'r') as f:
#             stored_codebook = json.load(f)

#     # calculate distortions

#     # print("User\tDistortion")
#     # print("----------------")
#     # with open('results.txt', 'a') as file:
#     #     file.write("User\tDistortion\n")
#     #     file.write("----------------\n")
#     user_distortion = {}
#     for user, cb in stored_codebook.items():
#         distortion = calculate_distortion(mfccs.T, cb)
#         # print(f"{user}\t{distortion}")
#         # with open('results.txt', 'a') as file:
#         #     file.write(f"{user}\t{distortion}\n")
#         if user not in user_distortion:
#             user_distortion[user] = distortion
#     min_key = min(user_distortion, key=user_distortion.get)
#     print(min_key)
#     print(user_distortion.get(min_key))
#     print("----------------")

#     # with open('results.txt', 'a') as file:
#     #     file.write("----------------\n")
#     #     file.write("----------------\n")
#     #     file.write("----------------\n")
# # with open('results.txt', 'a') as file:
# #     file.write('---------------------------------------\n')
# print("\n")
# print("\n")
# print("\n")


mfcc	20
codebook depth	32
Alanny.wav
Facundo
6719.823246838253
----------------
Facundo.wav
Facundo
5231.843252636605
----------------
Cesar.wav
Cesar
5561.206658838766
----------------
Katia.wav
Katia
6107.543385221244
----------------








# Plot the features and centroids

In [ ]:
# feature1_idx = 1  
# feature2_idx = 2

# plt.figure(figsize=(12, 8)) 

# for user, sample_mfcc in mfccs_per_user.items():
#     # Plot MFCC
#     #plt.scatter(sample_mfcc[feature1_idx,:], sample_mfcc[feature2_idx,:], label=user)
#     # Plot centroid
#     cb_stored=np.array(stored_codebook[user])
#     plt.scatter(cb_stored[:,feature1_idx], cb_stored[:,feature2_idx], label=f'Centroid of {user}', marker='x')
   
# plt.xlabel(f'Feature {feature1_idx+1}')
# plt.ylabel(f'Feature {feature2_idx+1}')
# plt.title(f'MFCC and Centroid')
# plt.legend()
# plt.grid(True)
# plt.savefig(f"mfcc_{num_mfcc}_cb_{cb_depth}.png")
# plt.close()
# plt.show()

In [ ]:
# ## MODIFICATIONS FOR TESTING
# sample_folder_rel='Sample'
# sample_folder= os.path.join(os.getcwd(), sample_folder_rel)
# sample_filename_rel='sample2.wav'
# sample_path= os.path.join(sample_folder, sample_filename_rel)
# # Read audio
# audio_file, sr = librosa.load(sample_path, sr=fs)

# # ## MODIFICATIONS FOR TESTING
# # sample_folder_rel='Users_samples'
# # sample_folder= os.path.join(os.getcwd(), sample_folder_rel)
# # sample_filename_rel='Cesar'
# # sample_folder= os.path.join(sample_folder, sample_filename_rel)
# # sample_filename_rel='voice_Cesar_1.wav'
# # sample_path= os.path.join(sample_folder, sample_filename_rel)
# # # Read audio
# # audio_file, sr = librosa.load(sample_path, sr=fs)

In [ ]:

# # entry = input("Press a key to enter a voice sample for identification: ")
# # audio_file, fs = record_audio(2, "", "")

# audio = audio_file.reshape(-1)

# num_mfcc = num_mfcc  # Number of MFCC coefficients
# frame_length = frame_lenght  # Frame length for STFT
# hop_length = hop_lenght  # Hop length for STFT

# mfccs = extract_mfcc(audio, fs, num_mfcc, frame_length, hop_length)

# print("Loading Codebook ... ")

# with open(os.path.join(codebook_folder, 'Codebook'), 'r') as f:
#         stored_codebook = json.load(f)

# # calculate distortions

# print("User\tDistortion")
# print("----------------")
# for user, cb in stored_codebook.items():
#     distortion = calculate_distortion(mfccs.T, cb)
#     print(f"{user}\t{distortion}")
    

